In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL.Image import Image as PilImage

import sagemaker

from sagemaker.serializers import JSONSerializer, IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

In [7]:
s3_bucket_name = sagemaker.session.Session().default_bucket()

In [9]:
!touch dummy
!tar czvf model.tar.gz dummy sagemaker-logo-small.png
assets_dir = 's3://{0}/{1}/assets/'.format(s3_bucket_name, 'image2text')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(s3_bucket_name, 'image2text')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
tar: sagemaker-logo-small.png: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors
upload: ./model.tar.gz to s3://sagemaker-us-west-2-691188012938/image2text/assets/model.tar.gz


In [1]:
!mkdir -p code

In [22]:
%%writefile code/inference.py

import io
import torch
from PIL import Image
import json
import logging
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# defining model and loading weights to it.
def model_fn(model_dir):
    model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    return {"model": model, "feature_extractor": feature_extractor,"tokenizer": tokenizer}

def load_from_bytearray(request_body):
    return image

# data loading
def input_fn(request_body, request_content_type):
    assert request_content_type in (
        "application/json",
        "application/x-image",
    ), f"{request_content_type} is an unknown type."
    if request_content_type == "application/json":
        data = json.loads(request_body)["inputs"]
    elif request_content_type == "application/x-image":
        image_as_bytes = io.BytesIO(request_body)
        data = Image.open(image_as_bytes)
    return data

# inference
def predict_fn(input_object, model):
    model_obj = model["model"]
    feature_extractor_obj = model["feature_extractor"]    
    tokenizer_obj = model["tokenizer"]       
    model_obj.to(device)
    max_length = 16
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
    images = []
    if input_object.mode != "RGB":
        input_object = input_object.convert(mode="RGB")
    images.append(input_object)
    pixel_values = feature_extractor_obj(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    output_ids = model_obj.generate(pixel_values, **gen_kwargs)
    preds = tokenizer_obj.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

# Serialize the prediction result into the desired response content type
def output_fn(predictions, content_type):
    assert content_type == "application/json"
    return json.dumps(predictions)

Overwriting code/inference.py


In [23]:
%%writefile code/requirements.txt
sentence-transformers

Overwriting code/requirements.txt


In [24]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session


role = get_execution_role()
shared_params = dict(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=f"s3://{s3_bucket_name}/image2text/assets/model.tar.gz",
    framework_version="1.13.1",
    py_version="py39",
)


image2text_model = PyTorchModel(
    name="image2text-model",
    **shared_params
)



In [25]:
text_predictor = image2text_model.deploy(
    instance_type='ml.c5.2xlarge',
    initial_instance_count=1,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    wait=True
)

Using already existing model: image2text-model


----!

In [26]:
import boto3
import json

# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-west-2')

def encode_image(file_name='./photos/_9dAwWA4LD8.jpg'):    
    with open(file_name, "rb") as f:
        payload = f.read()
        print(payload)
        payload = bytearray(payload)
    #res = image_predictor.predict(payload)
    res = sagemaker_runtime.invoke_endpoint(
                        EndpointName = "image2text-model-2023-08-03-13-01-37-053",
                        ContentType  = "application/x-image",
                        Body         = payload)
    res_final=json.loads((res['Body'].read()))
    
    return res_final[0]



In [30]:
feature_vector_text1 = encode_image('./photos/lyStEjlKNSw.jpg')

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x01\x00\x00\x0cHLino\x02\x10\x00\x00mntrRGB XYZ \x07\xce\x00\x02\x00\t\x00\x06\x001\x00\x00acspMSFT\x00\x00\x00\x00IEC sRGB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-HP  \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11cprt\x00\x00\x01P\x00\x00\x003desc\x00\x00\x01\x84\x00\x00\x00lwtpt\x00\x00\x01\xf0\x00\x00\x00\x14bkpt\x00\x00\x02\x04\x00\x00\x00\x14rXYZ\x00\x00\x02\x18\x00\x00\x00\x14gXYZ\x00\x00\x02,\x00\x00\x00\x14bXYZ\x00\x00\x02@\x00\x00\x00\x14dmnd\x00\x00\x02T\x00\x00\x00pdmdd\x00\x00\x02\xc4\x00\x00\x00\x88vued\x00\x00\x03L\x00\x00\x00\x86view\x00\x00\x03\xd4\x00\x00\x00$lumi\x00\x00\x03\xf8\x00\x00\x00\x14meas\x00\x00\x04\x0c\x00\x00\x00$tech\x00\x00\x040\x00\x00\x00\x0crTRC\x00\x00\x04<\x0

In [31]:
print(feature_vector_text1)

a black and white dog and a brown and white dog


In [ ]:

import boto3
import json

# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-west-2')

def encode_name(item_name):
    payload={"inputs": f"{item_name}"}
    payload_data=json.dumps(payload).encode('utf-8')
    res = sagemaker_runtime.invoke_endpoint(
                        EndpointName = "clip-text-model1-2023-07-10-08-56-15-444",
                        ContentType  = "application/json",
                        Body         = bytes(payload_data))
    res_final=json.loads((res['Body'].read()))
    return res_final[0]

In [ ]:
feature_vector_text = encode_name(feature_vector_text1)